In [1]:
from curtsies.fmtfuncs import red, green, blue, yellow
from cv2 import cv2
import numpy as np
import os
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as functional
import torch.optim as optim
import time

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
OUTPUTS = ['backhand_index_pointing_down',
           'backhand_index_pointing_right',
           'backhand_index_pointing_up',
           'call_me_hand',
           'crossed_fingers',
           'flexed_bicep',
           'hand_with_fingers_splayed',
           'index_pointing_up',
           'left_facing_fist',
           'love-you_gesture']

In [3]:
DEVICE = 'cuda:0'

MODEL_NAME = f"model-{int(time.time() % 100000)}"
IMAGE_SIZE_X = 200
IMAGE_SIZE_Y = 200

if torch.cuda.is_available():
    device = torch.device(DEVICE)
    print(green("Running on GPU"))
else:
    device = torch.device(DEVICE)
    print(yellow("Running on CPU"))

Running on GPU


In [4]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv2d(1, 32, 5)
        self.conv2 = nn.Conv2d(32, 64, 5)
        self.conv3 = nn.Conv2d(64, 128, 5)

        x = torch.randn(IMAGE_SIZE_X, IMAGE_SIZE_Y).view(-1, 1, IMAGE_SIZE_X, IMAGE_SIZE_Y)
        self._to_linear = None
        self.convs(x)

        self.fc1 = nn.Linear(self._to_linear, 512)
        self.fc2 = nn.Linear(512, 25)

    def convs(self, x):
        x = functional.max_pool2d(functional.relu(self.conv1(x)), (2, 2))
        x = functional.max_pool2d(functional.relu(self.conv2(x)), (2, 2))
        x = functional.max_pool2d(functional.relu(self.conv3(x)), (2, 2))

        if self._to_linear is None:
            self._to_linear = x[0].shape[0] * x[0].shape[1] * x[0].shape[2]

        return x

    def forward(self, x):
        x = self.convs(x)
        x = x.view(-1, self._to_linear)
        x = functional.relu(self.fc1(x))
        x = self.fc2(x)
        return functional.softmax(x, dim=1)

In [5]:
def fwd_pass(net, loss_function, optimizer, x, y, train=False):
    if train:
        net.zero_grad()
        optimizer.zero_grad()

    outputs = net(x)
    matches = [torch.argmax(i).item() == j for i, j in zip(outputs, y)]

    accuracy = matches.count(True) / len(y)
    loss = loss_function(outputs, y)

    if train:
        loss.backward()
        optimizer.step()

    return accuracy, loss

In [6]:
def test(net, loss_function, optimizer, test_x, test_y, size=50):
    rand_start = int(np.random.random() * (len(test_x) - size))
    test_x = test_x[rand_start:rand_start + size]
    test_y = test_y[rand_start:rand_start + size]
    batch_size = 50
    val_acc_sum = 0
    val_loss_sum = 0
    with torch.no_grad():
        for i in range(0, len(test_x), batch_size):
            batch_x = test_x[i:i + batch_size].view(-1, 1, IMAGE_SIZE_X, IMAGE_SIZE_Y).to(device)
            batch_y = test_y[i:i + batch_size].to(device)

            val_acc, val_loss = fwd_pass(net, loss_function, optimizer, batch_x, batch_y)
            val_acc_sum += val_acc
            val_loss_sum += val_loss.item()

    return val_acc_sum / (len(test_x) / batch_size), val_loss_sum / (len(test_x) / batch_size)

In [7]:
def train(net, loss_function, optimizer, train_x, train_y, test_x, test_y):
    batch_size = 100
    epochs = 5
    for epoch in range(epochs):
        count = 0
        val_count = 0
        sum_acc = 0
        sum_loss = 0
        sum_val_acc = 0
        sum_val_loss = 0
        for i in tqdm(range(0, len(train_x), batch_size)):
            batch_x = train_x[i:i + batch_size].view(-1, 1, IMAGE_SIZE_X, IMAGE_SIZE_Y).to(device)
            batch_y = train_y[i:i + batch_size].to(device)
            with torch.enable_grad():
                acc, loss = fwd_pass(net, loss_function, optimizer, batch_x, batch_y, train=True)
                sum_acc += acc
                sum_loss += loss.item()
                count += 1

            if i % 200 == 0:
                val_acc, val_loss = test(net, loss_function, optimizer, test_x, test_y)
                sum_val_acc += val_acc
                sum_val_loss += val_loss
                val_count += 1

        print('', flush=True)  # move to next line following tqdm's \r
        print(blue(f'EPOCH: {epoch}\t'
                     f'avg acc: {round(sum_acc / count, 5)}\t'
                     f'avg loss: {round(sum_loss / count, 5)}\t'
                     f'avg val acc: {round(sum_val_acc / val_count, 5)}\t'
                     f'avg val loss: {round(sum_val_loss / val_count, 5)}'), flush=True)

In [8]:
def loadTrainingData():
    training_data = np.load(os.path.join("gdrive", "MyDrive", "Hands_Emoji", "hands_10.npy"), allow_pickle=True)
    np.random.shuffle(training_data)

    x = torch.Tensor([i[0] for i in training_data]).view(-1, IMAGE_SIZE_X, IMAGE_SIZE_Y)
    x /= 255.  # Change pixel value to range[0, 1)
    y = [np.argwhere(i[1] == 1.)[0][0] for i in training_data]
    y = torch.LongTensor(y)

    VAL_PCT = 0.1
    val_size = int(len(x) * VAL_PCT)

    train_x = x[:-val_size]
    train_y = y[:-val_size]

    test_x = x[-val_size:]
    test_y = y[-val_size:]

    return train_x, train_y, test_x, test_y

In [9]:
# test image at gdrive/MyDrive/Hands_Emoji/HandEmojis/backhand_index_pointing_down/001.png
def test_image(net, name):
    _name = name
    while True:
        try:
            image = cv2.imread(_name, cv2.IMREAD_GRAYSCALE)
            break
        except FileNotFoundError as _:
            _name = str(input(red('could not find file. try again: ')))

    image = image[:, (1920 - 1080) // 2:(1920 - 1080) // 2 + 1080]

    resize_img = cv2.resize(image, (IMAGE_SIZE_X, IMAGE_SIZE_Y))

    resize_img = np.asarray(resize_img)
    resize_img = [i / 255. for i in resize_img]
    tensor_img = torch.tensor(resize_img, dtype=torch.float).view(-1, 1, IMAGE_SIZE_X, IMAGE_SIZE_Y)

    tensor_img = tensor_img.to(DEVICE)
    with torch.no_grad():
        output = net(tensor_img)

    argmax = int(torch.argmax(output))
    print(green(OUTPUTS[argmax]), '\tconfidence:', torch.max(output).item(), flush=True)
    print('', flush=True)

In [10]:
def main():

    print(red('creating network'), flush=True)
    net = Net().to(device)

    loss_function = nn.CrossEntropyLoss()
    optimizer = optim.Adam(net.parameters(), lr=.00001)

    print(yellow('loading training data'), flush=True)
    train_x, train_y, test_x, test_y = loadTrainingData()

    print(green('begin training\n'), flush=True)
    train(net, loss_function, optimizer, train_x, train_y, test_x, test_y)

    while True:
        choice = str(input('test with a picture filename or type exit: '))
        if choice == 'exit':
            break
        test_image(net, choice)

In [ ]:
if __name__=='__main__':
  main()